In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import numpy as np

In [2]:
df = pd.read_csv('reddit_preprocessing.csv')
df.shape

(36793, 2)

In [3]:
# Drop rows with NaN values in 'clean_comment'
cleaned_dataset = df.dropna()

# Separate features and target
X_cleaned = cleaned_dataset['clean_comment']
y_cleaned = cleaned_dataset['category']

In [4]:
# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

In [5]:
# Tokenize the sentences for Word2Vec training
X_train_tokenized = [sentence.split() for sentence in X_train_cleaned]
X_test_tokenized = [sentence.split() for sentence in X_test_cleaned]

In [6]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train_tokenized, vector_size=300, window=5, min_count=1, workers=4, sg=1)  # sg=1 for Skip-Gram model

In [7]:
# Generate the vector representation for each comment
def vectorize_comments(tokenized_comments, word2vec_model):
    vectorized_comments = []
    for tokens in tokenized_comments:
        vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
        if len(vectors) > 0:
            vectorized_comments.append(np.mean(vectors, axis=0))  # Mean of all word vectors in the comment
        else:
            vectorized_comments.append(np.zeros(word2vec_model.vector_size))  # If no words match, use a zero vector
    return np.array(vectorized_comments)

In [8]:
# Vectorize train and test comments
X_train_word2vec = vectorize_comments(X_train_tokenized, word2vec_model)
X_test_word2vec = vectorize_comments(X_test_tokenized, word2vec_model)

In [9]:
X_train_word2vec.shape

(29329, 300)

In [11]:
best_model = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric='multi_logloss',
    is_unblance=True,
    class_weight="balanced",
    reg_alpha=0.1,
    reg_lambda=0.1,
    learning_rate=0.077,
    max_depth=20,
    n_estimators=413
)

In [12]:
# Train the LightGBM model on Word2Vec embeddings
best_model.fit(X_train_word2vec, y_train_cleaned)

[LightGBM] [Warning] Unknown parameter: is_unblance
[LightGBM] [Warning] Unknown parameter: is_unblance
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014720 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 300
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


LGBMClassifier(class_weight='balanced', is_unblance=True, learning_rate=0.077,
               max_depth=20, metric='multi_logloss', n_estimators=413,
               num_class=3, objective='multiclass', reg_alpha=0.1,
               reg_lambda=0.1)

In [13]:
# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec)

[LightGBM] [Warning] Unknown parameter: is_unblance


/Users/mustainbillah/Projects/sentiment-analysis/venv-sentiment/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [14]:
print(classification_report(y_test_cleaned, y_pred))

              precision    recall  f1-score   support

          -1       0.49      0.50      0.50      1647
           0       0.73      0.73      0.73      2510
           1       0.69      0.68      0.69      3176

    accuracy                           0.66      7333
   macro avg       0.64      0.64      0.64      7333
weighted avg       0.66      0.66      0.66      7333



In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import numpy as np
import optuna
from sklearn.model_selection import cross_val_score

# Load the dataset
dataset = pd.read_csv('reddit_preprocessing.csv')

# Drop rows with NaN values in 'clean_comment'
cleaned_dataset = dataset.dropna()

# Separate features and target
X_cleaned = cleaned_dataset['clean_comment']
y_cleaned = cleaned_dataset['category']

# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(
    X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# Tokenize the sentences for Word2Vec training
X_train_tokenized = [sentence.split() for sentence in X_train_cleaned]
X_test_tokenized = [sentence.split() for sentence in X_test_cleaned]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train_tokenized, vector_size=100, window=5, min_count=1, workers=4, sg=1)  # sg=1 for Skip-Gram model

# Generate the vector representation for each comment
def vectorize_comments(tokenized_comments, word2vec_model):
    vectorized_comments = []
    for tokens in tokenized_comments:
        vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
        if len(vectors) > 0:
            vectorized_comments.append(np.mean(vectors, axis=0))  # Mean of all word vectors in the comment
        else:
            vectorized_comments.append(np.zeros(word2vec_model.vector_size))  # If no words match, use a zero vector
    return np.array(vectorized_comments)

# Vectorize train and test comments
X_train_word2vec = vectorize_comments(X_train_tokenized, word2vec_model)
X_test_word2vec = vectorize_comments(X_test_tokenized, word2vec_model)

# Encode the target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_cleaned)
y_test_encoded = label_encoder.transform(y_test_cleaned)

# Objective function for Optuna hyperparameter tuning
def objective(trial):
    # Suggest hyperparameters to be tuned
    params = {
        "objective": "multiclass",
        "num_class": 3,
        "metric": "multi_logloss",
        "is_unbalance": True,
        "class_weight": "balanced",
        "reg_alpha": 0.1,  # L1 regularization
        "reg_lambda": 0.1,  # L2 regularization
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 20)
    }

    # Initialize the LightGBM model with suggested parameters
    model = LGBMClassifier(**params)

    # Perform cross-validation to evaluate the model performance
    scores = cross_val_score(model, X_train_word2vec, y_train_encoded, cv=3, scoring='accuracy')

    # Return the mean accuracy score across folds
    return scores.mean()

# Create an Optuna study to optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # Run 30 trials

# Print the best trial
print("Best trial:")
best_params = study.best_trial.params
print(best_params)

# Train the LightGBM model with the best parameters
best_model = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    is_unbalance=True,
    class_weight="balanced",
    reg_alpha=0.1,
    reg_lambda=0.1,
    **best_params
)

best_model.fit(X_train_word2vec, y_train_encoded)

# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec)




[I 2025-06-06 10:02:24,364] A new study created in memory with name: no-name-b45440ab-14d8-45ba-b68b-d55d5e49e42f


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002813 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19552, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

/Users/mustainbillah/Projects/sentiment-analysis/venv-sentiment/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003498 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

/Users/mustainbillah/Projects/sentiment-analysis/venv-sentiment/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

/Users/mustainbillah/Projects/sentiment-analysis/venv-sentiment/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003136 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/Users/mustainbillah/Projects/sentiment-analysis/venv-sentiment/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-06-06 11:07:47,626] Trial 29 finished with value: 0.6540965864194338 and parameters: {'n_estimators': 838, 'learning_rate': 0.1488043256789763, 'max_depth': 15}. Best is trial 18 with value: 0.6546421994436659.


Best trial:
{'n_estimators': 638, 'learning_rate': 0.26074435615372027, 'max_depth': 10}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003925 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/Users/mustainbillah/Projects/sentiment-analysis/venv-sentiment/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [17]:
# Print classification report
print(classification_report(y_test_encoded, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.42      0.47      1647
           1       0.73      0.72      0.72      2510
           2       0.66      0.74      0.70      3176

    accuracy                           0.66      7333
   macro avg       0.64      0.63      0.63      7333
weighted avg       0.66      0.66      0.66      7333

